In [6]:
f = open("input.txt", "r").read()

grid = f.split("\n\n")[0].split("\n")
inputs = "".join(f.split("\n\n")[1].split("\n"))

rows = len(grid)
cols = len(grid[0])

for i in range(rows):
    for j in range(cols):
        if grid[i][j] == "@":
            robo = (i, j)

# grid, inputs, robo

In [7]:
def in_grid(pos, grid):
    return 0 <= pos[0] < len(grid) and 0 <= pos[1] < len(grid[0])

def is_wall(pos, grid):
    return grid[pos[0]][pos[1]] == "#"

def is_object(pos, grid):
    return grid[pos[0]][pos[1]] == "O"

def is_empty(pos, grid):
    return grid[pos[0]][pos[1]] == "."

def move(pos, new_pos, grid):
    char = grid[pos[0]][pos[1]]
    grid[new_pos[0]] = grid[new_pos[0]][:new_pos[1]] + char + grid[new_pos[0]][new_pos[1]+1:]
    grid[pos[0]] = grid[pos[0]][:pos[1]] + "." + grid[pos[0]][pos[1]+1:]
    
    return grid

def walk(pos, dir, grid):
    new_pos = (pos[0]+dir[0], pos[1]+dir[1])
    
    if not in_grid(new_pos, grid) or is_wall(new_pos, grid):
        return pos, grid
    
    if is_empty(new_pos, grid):
        grid = move(pos, new_pos, grid)
        return new_pos, grid
    
    if is_object(new_pos, grid):
        num_objects = 1
        object_pos = new_pos[:]
        
        # count objects
        while True:
            object_pos = (object_pos[0]+dir[0], object_pos[1]+dir[1])
            if not in_grid(object_pos, grid) or is_wall(object_pos, grid):
                return pos, grid
            if is_object(object_pos, grid):
                num_objects += 1
            if is_empty(object_pos, grid):
                break
        
        for _ in range(num_objects, 0, -1):
            tmp_pos = (object_pos[0]-dir[0], object_pos[1]-dir[1])
            grid = move(tmp_pos, object_pos, grid)
            object_pos = tmp_pos
        
        grid = move(pos, new_pos, grid)
        return new_pos, grid

dir_map = {
    "^": (-1, 0),
    "v": (1, 0),
    "<": (0, -1),
    ">": (0, 1),
}

for i in inputs:
    robo, grid = walk(robo, dir_map[i], grid)
    # print(f"Move {i}:")
    # for row in grid:
    #     print(row)
    
    
ans = 0
for i in range(rows):
    for j in range(cols):
        if grid[i][j] == "O":
            ans += 100*i+j
            
ans

1383666

In [8]:
f = open("input.txt", "r").read()

grid = f.split("\n\n")[0].split("\n")
inputs = "".join(f.split("\n\n")[1].split("\n"))

rows = len(grid)
cols = len(grid[0])

def replace(grid, pos, char):
    grid[pos[0]] = grid[pos[0]][:pos[1]] + char + grid[pos[0]][pos[1]+1:]
    return grid

for i in range(rows):
    for j in range(0,cols*2, 2):
        if grid[i][j] == ".":
            grid = replace(grid, (i, j), "..")
        elif grid[i][j] == "#":
            grid = replace(grid, (i, j), "##")
        elif grid[i][j] == "O":
            grid = replace(grid, (i, j), "[]")
        elif grid[i][j] == "@":
            grid = replace(grid, (i, j), "@.")

cols = len(grid[0])



for i in range(rows):
    for j in range(cols):
        if grid[i][j] == "@":
            robo = (i, j)


def in_grid(pos, grid):
    return 0 <= pos[0] < len(grid) and 0 <= pos[1] < len(grid[0])

def is_wall(pos, grid):
    return grid[pos[0]][pos[1]] == "#"

def is_object(pos, grid):
    return is_object_left(pos, grid) or is_object_right(pos, grid)

def is_object_left(pos, grid):
    return grid[pos[0]][pos[1]] == "["

def is_object_right(pos, grid):
    return grid[pos[0]][pos[1]] == "]"

def is_empty(pos, grid):
    return grid[pos[0]][pos[1]] == "."

def get_object(pos, grid):
    # always returns the left side of the object
    if not is_object(pos, grid):
        return None    
    if is_object_left(pos, grid):
        return pos
    if is_object_right(pos, grid):
        return (pos[0], pos[1]-1)

def can_move_object(pos, dir, grid):
    if dir[0] == 0:
        #move sideways, we dont need to check if move cascades
        new_pos = (pos[0], pos[1]+2*dir[1])
        
        return (is_empty(new_pos, grid) or 
                (not is_wall(new_pos, grid) and 
                can_move_object(new_pos, dir, grid)))
    else:
        #move up or down, we need to check if move cascades
        if is_object_left(pos, grid):
            pos_left = pos
            pos_right = (pos[0], pos[1]+1)
        if is_object_right(pos, grid):
            pos_left = (pos[0], pos[1]-1)
            pos_right = pos
        
        new_pos_right = (pos_right[0]+dir[0], pos_right[1]+dir[1])
        new_pos_left = (pos_left[0]+dir[0], pos_left[1]+dir[1])
        
        if is_empty(new_pos_left, grid) and is_empty(new_pos_right, grid):
            return True
        if is_wall(new_pos_left, grid) or is_wall(new_pos_right, grid):
            return False
        
        # blocked by at least one object, get blocking objects and check them:
        objects_to_move = set([get_object(new_pos_left, grid), get_object(new_pos_right, grid)])
        for obj in objects_to_move:
            if not obj is None and not can_move_object(obj, dir, grid):
                return False
        return True
        
def move_object(pos, dir, grid):
    
    if dir[0] == 0:
        #move sideways, we dont need to check if move cascades
        if is_object_left(pos, grid):
            pos_left = pos
        if is_object_right(pos, grid):
            pos_left = (pos[0], pos[1]-1)
            
        new_pos = (pos_left[0], pos_left[1]+dir[1])
        if dir[1] == 1:
            behind_object = (pos_left[0], pos_left[1]+2*dir[1])
        else:
            behind_object = (pos_left[0], pos_left[1]+dir[1])
        
        if is_empty(behind_object, grid):
            grid[pos_left[0]] = grid[pos_left[0]][:new_pos[1]] + "[]" + grid[pos_left[0]][new_pos[1]+2:]
            return grid
        if is_object(behind_object, grid):
            grid = move_object(behind_object, dir, grid)
            grid[pos_left[0]] = grid[pos_left[0]][:new_pos[1]] + "[]" + grid[pos_left[0]][new_pos[1]+2:]
            return grid
        
    else:
        #move up or down, we need to check if move cascades
        if is_object_left(pos, grid):
            pos_left = pos
            pos_right = (pos[0], pos[1]+1)
        if is_object_right(pos, grid):
            pos_left = (pos[0], pos[1]-1)
            pos_right = pos
        
        new_pos_right = (pos_right[0]+dir[0], pos_right[1]+dir[1])
        new_pos_left = (pos_left[0]+dir[0], pos_left[1]+dir[1])
        
        if is_empty(new_pos_left, grid) and is_empty(new_pos_right, grid):
            grid[new_pos_left[0]] = grid[new_pos_left[0]][:new_pos_left[1]] + "[]" + grid[new_pos_left[0]][new_pos_left[1]+2:]
            grid[pos_left[0]] = grid[pos_left[0]][:pos_left[1]] + ".." + grid[pos_left[0]][pos_left[1]+2:]
            return grid
        
        # blocked by at least one object, get blocking object:
        objects_to_move = set([get_object(new_pos_left, grid), get_object(new_pos_right, grid)])
        
        for obj in objects_to_move:
            if not obj is None:
                grid = move_object(obj, dir, grid)
        
        grid[new_pos_left[0]] = grid[new_pos_left[0]][:new_pos_left[1]] + "[]" + grid[new_pos_left[0]][new_pos_left[1]+2:]
        grid[pos_left[0]] = grid[pos_left[0]][:pos_left[1]] + ".." + grid[pos_left[0]][pos_left[1]+2:]
        
    return grid

def move(pos, dir, grid):
    new_pos = (pos[0]+dir[0], pos[1]+dir[1])
    
    if is_wall(new_pos, grid):
        # do nothing
        return pos, grid
    
    if is_empty(new_pos, grid):
        char = grid[pos[0]][pos[1]]
        grid[new_pos[0]] = grid[new_pos[0]][:new_pos[1]] + char + grid[new_pos[0]][new_pos[1]+1:]
        grid[pos[0]] = grid[pos[0]][:pos[1]] + "." + grid[pos[0]][pos[1]+1:]
        return new_pos, grid
    
    # not wall, not empty -> is object
    # try to move object
    if can_move_object(new_pos, dir, grid):
        grid = move_object(new_pos, dir, grid)
        char = grid[pos[0]][pos[1]]
        grid[new_pos[0]] = grid[new_pos[0]][:new_pos[1]] + char + grid[new_pos[0]][new_pos[1]+1:]
        grid[pos[0]] = grid[pos[0]][:pos[1]] + "." + grid[pos[0]][pos[1]+1:]

        return new_pos, grid
    
    return pos, grid

dir_map = {
    "^": (-1, 0),
    "v": (1, 0),
    "<": (0, -1),
    ">": (0, 1),
}

for i in inputs:
    robo, grid = move(robo, dir_map[i], grid)
    # print(f"Move {i}:")
    # for row in grid:
    #     print(row)
    
    
ans = 0
for i in range(rows):
    for j in range(cols):
        if grid[i][j] == "[":
            ans += 100*i+j
            
ans

1412866